<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **COVID Dashboard**

## 1\. Contexto

 - **Dashboard**:
  - Google Data Studio (https://lookerstudio.google.com/s/nlej-12I39o).
 - **Processamento**:
  - Kaggle Notebook (https://www.kaggle.com/lucasguimabarrroso/covid-19-analise).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

O estudo tem como objetivo analisar dados de contágio e de vacinação em relação a pandemia da COVID-19. Os dados de vacinação estão até o ano de 2024 enquanto os dados de contágio estão somente até o ano de 2023. Para uma melhor análise faremos o estudo no período de janeiro de 2022 a dezembro de 2022, pois os dados do ano de 2023 estão incompletos.

### **Dados**

Os dados relacionados ao contágio foram compilados pelo centro de ciência de sistemas e engenharia da universidade americana **John hopkins** ([link](https://www.jhu.edu)). A granularidade temporal dos dados está em dias e geográfica, de regiões de países e são atualizados diariamente. O website do projeto encontra-se neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

Os dados sobre vacinação são copilados pelo projeto Nosso Mundo em Dados (*Our world in Data* ou OWID)da universidade britânica de Oxford ([link](https://www.ox.ac.uk)). Os dados são atualizados diariamente e estão numa granularidade temporal de dias e geográfica de países. O site do projeto pode ser acessado por meio detse [link](https://ourworldindata.org) e os dados por meio deste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados de seu processamento.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## Pacotes e bibliotecas

Neste estudo utilizaremos as seguintes bibliotecas Python para a análise exploratória dos dados.

In [177]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## Extração - casos



Abaixo o código de extração dos dados referente aos casos de COVID-19.

Os dados estão compilados em um arquivo por dia.

In [178]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

os dados estão divididos nas seguintes colunas como vemos abaixo. Porém só utilizaremos as colunas Province_state, Country_Region, Confirmed, Deaths e Incident_Rate.

In [179]:
cases.head(1)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192


Os dados estão separados diariamente em vários arquivos, portanto, será necessário fazer a extração do conteúdo diário em cada arquivo e compilar tudo num único dado. Para isso utilizaremos o código abaixo que coleta a informação de todas as datas no intervalo que foi passado, busca o arquivo referente aquela data e salva os dados em um data frame pandas.

In [180]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [181]:
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

In [182]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case["Date"] = pd.to_datetime(date.strftime('%Y-%m-%d'))


  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], ignore_index=True)


In [183]:
cases.head() # verifica as primeiras linhas dos dados após a coleta

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,88386,1851,10021.827005,2022-01-01
1,Alagoas,Brazil,242091,6383,7253.973728,2022-01-01
2,Amapa,Brazil,127013,2022,15018.132243,2022-01-01
3,Amazonas,Brazil,433850,13836,10467.845245,2022-01-01
4,Bahia,Brazil,1271029,27507,8545.845026,2022-01-01


In [184]:
cases.shape # verificar a quantidade de linhas e de colunas no Data Frame

(9828, 6)

In [185]:
cases.info() # verifica os tipos dos dados e se há valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


## Transformação - casos

In [186]:
# mudança no nome das colunas para melhor entendimento

# primeiro mudaremos as colunas Province_State e Country_Region para state e country, respectivamente.
cases = cases.rename(
    columns={
        'Province_State':'state',
        'Country_Region':'country'
    }
)

# depois deixaremos o nome de todas as colunas em letras minúsculas para padronização.
for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [187]:
cases.head(1)

,state,country,confirmed,deaths,incident_rate,date
0,Acre,Brazil,88386,1851,10021.827005,2022-01-01


In [188]:
# usando função lambda para mudar o nome dos estados para o nome correto com a devida acentuação

cases['state'] = cases['state'].apply(lambda state: 'Amapá' if state == 'Amapa' else state)
cases['state'] = cases['state'].apply(lambda state: 'Ceará' if state == 'Ceara' else state)
cases['state'] = cases['state'].apply(lambda state: 'Espírito Santo' if state == 'Espirito Santo' else state)
cases['state'] = cases['state'].apply(lambda state: 'Goiás' if state == 'Goias' else state)
cases['state'] = cases['state'].apply(lambda state: 'Pará' if state == 'Para' else state)
cases['state'] = cases['state'].apply(lambda state: 'Paraíba' if state == 'Paraiba' else state)
cases['state'] = cases['state'].apply(lambda state: 'Paraná' if state == 'Parana' else state)
cases['state'] = cases['state'].apply(lambda state: 'Piauí' if state == 'Piaui' else state)
cases['state'] = cases['state'].apply(lambda state: 'Rondônia' if state == 'Rondonia' else state)
cases['state'] = cases['state'].apply(lambda state: 'São Paulo' if state == 'Sao Paulo' else state)
cases['state'] = cases['state'].apply(lambda state: 'Maranhão' if state == 'Maranhao' else state)


In [189]:
cases.head(28)

,state,country,confirmed,deaths,incident_rate,date
0,Acre,Brazil,88386,1851,10021.827005,2022-01-01
1,Alagoas,Brazil,242091,6383,7253.973728,2022-01-01
2,Amapá,Brazil,127013,2022,15018.132243,2022-01-01
3,Amazonas,Brazil,433850,13836,10467.845245,2022-01-01
4,Bahia,Brazil,1271029,27507,8545.845026,2022-01-01
5,Ceará,Brazil,957035,24806,10479.925818,2022-01-01
6,Distrito Federal,Brazil,519811,11108,17239.296805,2022-01-01
7,Espírito Santo,Brazil,629925,13330,15675.040125,2022-01-01
8,Goiás,Brazil,947907,24695,13506.115536,2022-01-01
9,Maranhão,Brazil,370694,10377,5239.357127,2022-01-01


- Acrescentando duas colunas de chaves temporais com mês e ano.

In [190]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year'] = cases['date'].apply(lambda date: date.strftime('%Y'))

- Acrescentando uma coluna com a população estimada de cada estado

In [191]:
# a coluna 'incident_rate' traz a informação de quantos casos acontecem a cada 100000 pessoas
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))

cases = cases.drop('incident_rate', axis=1)

O código abaixo cria as colunas com os dados de confirmados e mortos por dia, média dos ultimos 7 dias, comparação dos dados com 14 dias de diferença e tendência.

A função get_trend é aplicada a partir do valor da coluna 'confirmed_moving_avg_7d_rate_14d', essa função nos mostra se depois de 14 dias os dados estão estáveis, subindo ou decaindo.

O laço *for* itera sobre o nome de cada estado, retirando as repetições e então salva em um data frame chamado "cases_per_state' ordenando por data as infomações de um estado por vez.

A partir disso é realizado o tratamento dos dados referente aos casos confirmados e números de mortos:
- moving_avg_7d: Média dos últimos 7 dias utilizando .rolling(window=7).mean() e então np.ceil para arredondar esse valor.
- moving_avg_7d_rate_14d: Utiliza a informação da média dos ultimos 7 dias e divide pelo mesmo valor porém de 14 dias atrás, para chegar nesse dado de 14 dias atrás foi utilizado .shift(periods=14).
- trend: A partir das informações das colunas moving_avg_7d_rate_14d a função get_trend é aplicada e então nos retorna se aquele dado é *downward*, *upward* ou *stable*

todos esses dados são salvos na variável cases_ e depois tranferidos para o df original **cases**


In [192]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.85:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_1d'] = cases_per_state['confirmed_1d'].apply(lambda cases: 0 if cases < 0 else cases)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_1d'] = cases_per_state['deaths_1d'].apply(lambda deaths: 0 if deaths < 0 else deaths)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)


  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False

  else:
    cases_ = pd.concat([cases_, cases_per_state], ignore_index=True)

cases = cases_
cases_ = None

In [193]:
cases.head()

,state,country,confirmed,deaths,date,month,year,population,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend
0,Acre,Brazil,88386,1851,2022-01-01,2022-01,2022,881935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acre,Brazil,88386,1851,2022-01-02,2022-01,2022,881935.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,Acre,Brazil,88395,1851,2022-01-03,2022-01,2022,881935.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,Acre,Brazil,88395,1852,2022-01-04,2022-01,2022,881935.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,Acre,Brazil,88395,1852,2022-01-05,2022-01,2022,881935.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [194]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   state                             9828 non-null   object        
 1   country                           9828 non-null   object        
 2   confirmed                         9828 non-null   int64         
 3   deaths                            9828 non-null   int64         
 4   date                              9828 non-null   datetime64[ns]
 5   month                             9828 non-null   object        
 6   year                              9828 non-null   object        
 7   population                        9828 non-null   float64       
 8   confirmed_1d                      9801 non-null   float64       
 9   confirmed_moving_avg_7d           9639 non-null   float64       
 10  confirmed_moving_avg_7d_rate_14d  9261 non-null 

*Type casting* das colunas population, confirmed_1d, confirmed_moving_avg_7d, deaths_1d e deaths_moving_avg_7d para os tipos de dados 'Int64'. Esse tipo de dado foi escolhido pois ele suporta tratar dados do tipo NaN, diferente do 'int' que não reconhece esse tipo de dado nulo.

In [195]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [196]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   state                             9828 non-null   object        
 1   country                           9828 non-null   object        
 2   confirmed                         9828 non-null   int64         
 3   deaths                            9828 non-null   int64         
 4   date                              9828 non-null   datetime64[ns]
 5   month                             9828 non-null   object        
 6   year                              9828 non-null   object        
 7   population                        9828 non-null   Int64         
 8   confirmed_1d                      9801 non-null   Int64         
 9   confirmed_moving_avg_7d           9639 non-null   Int64         
 10  confirmed_moving_avg_7d_rate_14d  9261 non-null 

Reorganização dos dados.

In [197]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [198]:
cases.head(25)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2022-01-01,Brazil,Acre,881935,88386,<NA>,<NA>,NaN,NaN,1851,<NA>,<NA>,NaN,NaN,2022-01,2022
1,2022-01-02,Brazil,Acre,881935,88386,0,<NA>,NaN,NaN,1851,0,<NA>,NaN,NaN,2022-01,2022
2,2022-01-03,Brazil,Acre,881935,88395,9,<NA>,NaN,NaN,1851,0,<NA>,NaN,NaN,2022-01,2022
3,2022-01-04,Brazil,Acre,881935,88395,0,<NA>,NaN,NaN,1852,1,<NA>,NaN,NaN,2022-01,2022
4,2022-01-05,Brazil,Acre,881935,88395,0,<NA>,NaN,NaN,1852,0,<NA>,NaN,NaN,2022-01,2022
5,2022-01-06,Brazil,Acre,882015,88403,8,<NA>,NaN,NaN,1852,0,<NA>,NaN,NaN,2022-01,2022
6,2022-01-07,Brazil,Acre,882015,88403,0,<NA>,NaN,NaN,1852,0,<NA>,NaN,NaN,2022-01,2022
7,2022-01-08,Brazil,Acre,882015,88403,0,3,NaN,NaN,1852,0,1,NaN,NaN,2022-01,2022
8,2022-01-09,Brazil,Acre,881935,88403,0,3,NaN,NaN,1852,0,1,NaN,NaN,2022-01,2022
9,2022-01-10,Brazil,Acre,881935,88440,37,7,NaN,NaN,1852,0,1,NaN,NaN,2022-01,2022


In [199]:
df_filtrado = cases[(cases['date'] >= '2022-05-15') & (cases['date'] <= '2022-05-30')].reset_index(drop=True)
df_filtrado.tail(50)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
382,2022-05-29,Brazil,Santa Catarina,7164788,1738012,637,1834,1.525790,upward,21846,3,4,1.000000,stable,2022-05,2022
383,2022-05-30,Brazil,Santa Catarina,7164788,1738492,480,1903,1.599160,upward,21850,4,5,1.666667,upward,2022-05,2022
384,2022-05-15,Brazil,São Paulo,45919049,5449941,2336,3639,0.890384,stable,168710,41,53,1.656250,upward,2022-05,2022
385,2022-05-16,Brazil,São Paulo,45919049,5451144,1203,3741,0.916912,stable,168712,2,54,1.687500,upward,2022-05,2022
386,2022-05-17,Brazil,São Paulo,45919049,5451144,0,3064,0.725722,downward,168712,0,43,1.387097,upward,2022-05,2022
387,2022-05-18,Brazil,São Paulo,45919049,5451144,0,2387,0.555892,downward,168712,0,36,1.333333,upward,2022-05,2022
388,2022-05-19,Brazil,São Paulo,45919049,5451144,0,1741,0.390096,downward,168712,0,26,1.040000,stable,2022-05,2022
389,2022-05-20,Brazil,São Paulo,45919049,5459629,8485,2248,0.472071,downward,168864,152,36,1.714286,upward,2022-05,2022
390,2022-05-21,Brazil,São Paulo,45919049,5459629,0,1718,0.364524,downward,168864,0,28,1.647059,upward,2022-05,2022
391,2022-05-22,Brazil,São Paulo,45919049,5480732,21103,4399,0.950108,stable,169053,189,49,2.882353,upward,2022-05,2022


In [200]:
df_filtrado_mortes = cases[(cases['deaths_1d'] <0)].reset_index(drop=True)
df_filtrado_mortes.head()

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year


## Carregamento - casos

Carregamento dos dados para o arquivo covid-cases.csv

In [201]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

## Extração - vacinação

Extração dos dados de vacinação utilizando a coluna 3 como referência de data.

In [202]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

In [203]:
vaccines.head(1)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [204]:
vaccines.tail(1)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
373456,ZWE,Africa,Zimbabwe,2024-01-21,266257.0,55.0,7.857,5737.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN


In [205]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [206]:
vaccines.tail(50)

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
1428,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-03
1429,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-04
1430,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-05
1431,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-06
1432,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-07
1433,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-08
1434,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-09
1435,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-10
1436,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-11
1437,Brazil,215313504.0,NaN,NaN,NaN,NaN,2023-12-12


In [207]:
vaccines.shape

(1478, 7)

## Transformação - vacinação

In [208]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1478 non-null   object        
 1   population               1478 non-null   float64       
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1478 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 81.0+ KB


Tratamento dos valores nulos será com o método .fillna(method='ffill') que copia o dado anterior no dato que está nulo.

In [209]:
vaccines = vaccines.fillna(method='ffill')

In [210]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1478 non-null   object        
 1   population               1478 non-null   float64       
 2   total_vaccinations       1100 non-null   float64       
 3   people_vaccinated        1100 non-null   float64       
 4   people_fully_vaccinated  1081 non-null   float64       
 5   total_boosters           872 non-null    float64       
 6   date                     1478 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 81.0+ KB


Filtrando os dados com o mesmo período de tempo dos dados referente a contágio e mortes.

In [211]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2023-01-01')].reset_index(drop=True)

In [212]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-01
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-02
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-03
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-04
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-05


In [213]:
vaccines.tail()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
726,Brazil,215313504.0,480310839.0,188549744.0,174881292.0,122616211.0,2022-12-28
727,Brazil,215313504.0,480331769.0,188552661.0,174886102.0,122629436.0,2022-12-29
728,Brazil,215313504.0,480332769.0,188553047.0,174886846.0,122629436.0,2022-12-30
729,Brazil,215313504.0,480333910.0,188553932.0,174887915.0,122629436.0,2022-12-31
730,Brazil,215313504.0,480333982.0,188553963.0,174887956.0,122629436.0,2023-01-01


In [214]:
vaccines = vaccines.rename(
    columns={
        'location': 'country',
        'total_vaccinations': 'total',
        'people_vaccinated': 'one_shot',
        'people_fully_vaccinated': 'two_shots',
        'total_boosters': 'three_shots'
    }
)

Acrescentando colunas com informação de 'mês' e 'ano' para melhorar os dados.

In [215]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year'] = vaccines['date'].apply(lambda date: date.strftime('%Y'))

Acrescentando colunas com dados percentuais sobre as doses de vacinação.

In [216]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

*Type casting* das colunas population, total, one_shot, two_shots e three_shots para os tipos de dados 'Int64'. Esse tipo de dado foi escolhido pois ele suporta tratar dados do tipo NaN, diferente do 'int' que não reconhece esse tipo de dado nulo.

In [217]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [218]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [219]:
vaccines.tail()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
726,2022-12-28,Brazil,215313504,480310839,188549744,0.8757,174881292,0.8122,122616211,0.5695,2022-12,2022
727,2022-12-29,Brazil,215313504,480331769,188552661,0.8757,174886102,0.8122,122629436,0.5695,2022-12,2022
728,2022-12-30,Brazil,215313504,480332769,188553047,0.8757,174886846,0.8122,122629436,0.5695,2022-12,2022
729,2022-12-31,Brazil,215313504,480333910,188553932,0.8757,174887915,0.8122,122629436,0.5695,2022-12,2022
730,2023-01-01,Brazil,215313504,480333982,188553963,0.8757,174887956,0.8122,122629436,0.5695,2023-01,2023


## Carregamento - vacinação

In [220]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

### **KPIs**

O dashboard apresentado contém os seguintes indicadores chaves de desempenho (*key performance indicator* ou KPI) consolidados:
- Casos e mortes nas 24 horas;
- Média móvel (7 dias) de casos e mortes;
- Tendência de casos e mortes;
- Proporção de vacinados com 1ª, 2ª e 3ª doses.